
# 📒 Kraken Training — **ALTO-only** (Colab, live logs, smoke test included)

**What you get**
- Upload ALTO export ZIP(s) → auto-extracted
- Auto-discover ALTO XMLs (case-insensitive)
- **Smoke test** (1 epoch) to verify wiring
- Full **recognition** and **segmentation** training with **live logs**
- Evaluation + packaging
- Optional upload to **msia.escriptorium.fr**

> In Colab: **Runtime → Change runtime type → GPU** (recommended).


In [ ]:

# 0) GPU check
!nvidia-smi || true


## 1) Install Kraken (Py 3.12 safe) + deps

In [ ]:

import sys, subprocess
def pip_install(*pkgs):
    print("pip install", " ".join(pkgs))
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

# Colab-friendly pin
KR_PIN = "kraken==5.3.0"
pip_install(KR_PIN, "torch>=2.1,<3", "cairocffi", "opencv-python", "lxml", "h5py")


In [ ]:

from importlib.metadata import version, PackageNotFoundError
import torch, shutil, sys
def pkg_ver(name):
    try: return version(name)
    except PackageNotFoundError: return "not installed"
print("python:", sys.version.split()[0])
print("kraken:", pkg_ver("kraken"))
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("ketos path:", shutil.which("ketos"))
!ketos --version || true


## 2) Config — ALTO fixed, validation split, optional Drive roots

In [ ]:

# ======= CONFIG (ALTO) =======
FORMAT = "alto"     # hard-coded
VAL_FRACTION = 0.10 # change if you want
# Optional: add Drive roots if you also keep ALTO data on Drive
DRIVE_ROOTS = [
    # "/content/drive/MyDrive/my_alto_export_folder"
]
# =============================


### (Optional) Mount Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


## 3) Upload ALTO ZIP(s) — auto-extract and discover roots

In [ ]:

import os, io, zipfile, re
from google.colab import files

LOCAL_BASE = "/content/data_alto"
os.makedirs(LOCAL_BASE, exist_ok=True)

print("Upload ALTO export ZIP(s). They will be extracted into", LOCAL_BASE)
uploaded = files.upload()

EXTRACTED_ROOTS = []
for name, data in uploaded.items():
    path = os.path.join(LOCAL_BASE, name)
    with open(path, "wb") as f: f.write(data)
    if name.lower().endswith(".zip"):
        # strip any "(1)" suffix and .zip
        base = re.sub(r"\s*\(\d+\)\s*$", "", os.path.splitext(name)[0])
        target = os.path.join(LOCAL_BASE, base)
        os.makedirs(target, exist_ok=True)
        with zipfile.ZipFile(io.BytesIO(data)) as zf:
            zf.extractall(target)
        os.remove(path)
        EXTRACTED_ROOTS.append(target)
    else:
        EXTRACTED_ROOTS.append(LOCAL_BASE)

print("Extracted roots:")
for r in EXTRACTED_ROOTS: print(" -", r)


## 4) Build merged ALTO XML list + quick sanity checks

In [ ]:

import os, glob, xml.etree.ElementTree as ET

ROOTS = [p for p in (EXTRACTED_ROOTS + DRIVE_ROOTS) if p and os.path.exists(p)]
assert ROOTS, "No roots found. Upload a ZIP or set DRIVE_ROOTS and re-run."

def find_alto_xmls(root):
    xmls = glob.glob(os.path.join(root, "**", "*.xml"), recursive=True) +            glob.glob(os.path.join(root, "**", "*.XML"), recursive=True)
    out = []
    for xp in xmls:
        try:
            r = ET.parse(xp).getroot()
            if isinstance(r.tag, str) and "alto" in r.tag.lower():
                out.append(xp)
        except ET.ParseError:
            pass
    return sorted(out)

xmls = []
for root in ROOTS:
    found = find_alto_xmls(root)
    print(f"[ALTO] {len(found):>5} XMLs in {root}")
    xmls.extend(found)

print("TOTAL ALTO XML files:", len(xmls))
assert len(xmls) >= 3, "Need at least a few ALTO XML pages."

# Show a few
for p in xmls[:5]: print(" •", p)


## 5) Create train/val lists

In [ ]:

import random, pathlib
random.seed(42)

random.shuffle(xmls)
n_val = max(1, int(len(xmls)*VAL_FRACTION))
val_xmls = xmls[:n_val]
train_xmls = xmls[n_val:]

pathlib.Path("/content/lists").mkdir(parents=True, exist_ok=True)
open("/content/lists/train.txt","w").write("\n".join(train_xmls))
open("/content/lists/val.txt","w").write("\n".join(val_xmls))

print("Train pages:", len(train_xmls), "| Val pages:", len(val_xmls))
print("First train:", train_xmls[:2])
print("First val:", val_xmls[:2])


## 6) Hyperparameters (+ auto device) — change LR here

In [ ]:

# ======= EDIT ME (Hyperparameters) =======
import torch, os, shutil

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)
KETOS = shutil.which("ketos") or "ketos"
print("ketos path:", KETOS)

# Recognition
REC_EPOCHS = 30
REC_BATCH  = 8
REC_OPTIM  = "Adam"    # or "SGD"
REC_LR     = 0.0001    # <-- change learning rate here
REC_WD     = 1e-5

# Segmentation
SEG_EPOCHS = 20
SEG_BATCH  = 2
SEG_OPTIM  = "Adam"
SEG_LR     = 0.0005    # <-- change learning rate here
# =========================================


### (Optional) Peek at lists

In [ ]:

!sed -n '1,5p' /content/lists/train.txt
!sed -n '1,5p' /content/lists/val.txt


## 7) 🔎 Smoke test — recognition (1 epoch, small batch, live logs)

In [ ]:

%%bash
set -euo pipefail
which ketos || true
stdbuf -oL -eL ketos train -v   -f alto @/content/lists/train.txt   -o /content/models/rec_smoke   --device cpu   --epochs 1   --batch-size 2   --optimizer Adam --lr 0.0001 --weight-decay 1e-5   --validation @/content/lists/val.txt


## 8) ▶️ Recognition — full training (live logs)

In [ ]:

%%bash
set -euo pipefail
which ketos
stdbuf -oL -eL ketos train -v   -f alto @/content/lists/train.txt   -o /content/models/rec   --device ${DEVICE:-cpu}   --epochs ${REC_EPOCHS:-30}   --batch-size ${REC_BATCH:-8}   --optimizer ${REC_OPTIM:-Adam} --lr ${REC_LR:-0.0001} --weight-decay ${REC_WD:-1e-5}   --validation @/content/lists/val.txt


## 9) ▶️ Segmentation — full training (live logs)

In [ ]:

%%bash
set -euo pipefail
which ketos
stdbuf -oL -eL ketos segtrain -v   -f alto @/content/lists/train.txt   -o /content/models/seg   --device ${DEVICE:-cpu}   --epochs ${SEG_EPOCHS:-20}   --batch-size ${SEG_BATCH:-2}   --optimizer ${SEG_OPTIM:-Adam} --lr ${SEG_LR:-0.0005}   --validation @/content/lists/val.txt


## 10) Evaluate recognition (CER/WER)

In [ ]:

%%bash
set -euo pipefail
ketos test -f alto /content/models/rec_best.mlmodel @/content/lists/val.txt


## 11) Package models for download

In [ ]:

!cd /content/models && ls -lh && zip -r ../trained_models.zip . && cd /content


## 12) Optional — Upload to msia.escriptorium.fr via API

In [ ]:

# UI upload (My Models → Upload) is simplest.
# Programmatic upload (if supported by your instance):
MSIA_URL   = "https://msia.escriptorium.fr"
API_TOKEN  = "PASTE_YOUR_TOKEN_HERE"   # keep secret or leave blank
MODEL_PATH = "/content/models/rec_best.mlmodel"   # or seg_best.mlmodel
MODEL_NAME = "rec_best"

if API_TOKEN and API_TOKEN != "PASTE_YOUR_TOKEN_HERE":
    import requests
    headers = {"Authorization": f"Token {API_TOKEN}"}
    files = {"file": (MODEL_NAME + ".mlmodel", open(MODEL_PATH, "rb"), "application/octet-stream")}
    data  = {"name": MODEL_NAME}
    resp = requests.post(f"{MSIA_URL}/api/models/", headers=headers, files=files, data=data)
    print("Status:", resp.status_code)
    try:
        print(resp.json())
    except Exception:
        print(resp.text[:800])
else:
    print("Skipping API upload. Use the UI or paste a valid API token.")
